In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext


spark=SparkSession.builder.appName("Assignment_1").getOrCreate()

#products_df=spark.read.format("csv").options(header=True,inferSchema=True).load("/FileStore/tables/Production_Product.csv").select("ProductId","Name")
salesOrderDetail_df=spark.read.format("csv").options(header=True,inferSchema=True).load("/FileStore/tables/Sales_SalesOrderDetail.csv").select("SalesOrderID","OrderQty","LineTotal","ModifiedDate")
territory_df=spark.read.format("csv").options(header=True,inferSchema=True).load("/FileStore/tables/Sales_SalesTerritory.csv").select("TerritoryID","CountryRegionCode")
salesOrderHeader_df=spark.read.format("csv").options(header=True,inferSchema=True).load("/FileStore/tables/Sales_SalesOrderHeader.csv").select("SalesOrderID","ModifiedDate","TerritoryID")

In [2]:
from pyspark.sql.functions import *
salesOrderFiltered_df=salesOrderDetail_df.withColumn("year",year(salesOrderDetail_df["ModifiedDate"])).drop("ModifiedDate").filter(col("year").isin([2003,2004]))
#salesOrderFiltered_df.show(10,truncate=False)

#filter(("year" == 2003) | ("year" == 2004)).drop("ModifiedDate")
salesOrderHeaderFiltered_df=salesOrderHeader_df.withColumn("year",year(salesOrderHeader_df["ModifiedDate"])).drop("ModifiedDate").filter(col("year").isin([2003,2004]))
#salesOrderHeaderFiltered_df.show(10,truncate=False)

salesOrderHeaderFiltered_df.select("year").distinct().show()

+----+
year|
+----+
2004|
2003|
+----+

In [3]:
# Registering Temporary Tables
#products_df.createOrReplaceTempView("products")
salesOrderFiltered_df.createOrReplaceTempView("salesOrderDetail")
territory_df.createOrReplaceTempView("territory")
salesOrderHeaderFiltered_df.createOrReplaceTempView("SalesOrderHeader")

In [4]:
#spark.sql("select year, count(*) from  SalesOrderHeader group by year").show()
spark.sql("select count(*)  from  salesOrderDetail where year=2003").show()

+--------+
count(1)|
+--------+
 51237|
+--------+

In [5]:
spark.conf.set("spark.sql.shuffle.partitions",8)
agg_sales_2003=spark.sql("select /*+REPARTITION(8) */ t.CountryRegionCode as Country, d.Year as Year,SUM(d.OrderQty) as total_quantity,ROUND(CAST(SUM(d.LineTotal) as double)/1000000,3) as total_sales_in_million from territory t  \
                      INNER JOIN SalesOrderHeader h \
                  ON t.TerritoryID=h.TerritoryID \
                      INNER JOIN SalesOrderDetail d \
                  ON h.SalesOrderID=d.SalesOrderID  \
                    and h.year=d.year \
                        where d.Year=2003 GROUP BY t.CountryRegionCode,d.Year ")

agg_sales_2004=spark.sql("select /*+REPARTITION(8) */ t.CountryRegionCode as Country, d.Year as Year,SUM(d.OrderQty) as total_quantity,ROUND(CAST(SUM(d.LineTotal) as double)/1000000,3) as total_sales_in_million from territory t  \
                      INNER JOIN SalesOrderHeader h \
                  ON t.TerritoryID=h.TerritoryID \
                      INNER JOIN SalesOrderDetail d \
                  ON h.SalesOrderID=d.SalesOrderID \
                     and h.year=d.year \
                   where  d.Year=2004 GROUP BY t.CountryRegionCode,d.Year ")

In [7]:
agg_sales_2003.createOrReplaceTempView("agg_sales_2003")
agg_sales_2004.createOrReplaceTempView("agg_sales_2004")

spark.sql("select * from agg_sales_2004").show()

+-------+----+--------------+----------------------+
Country|Year|total_quantity|total_sales_in_million|
+-------+----+--------------+----------------------+
 DE|2004| 6016| 1.982|
 CA|2004| 12090| 3.064|
 FR|2004| 6928| 2.299|
 AU|2004| 8696| 3.311|
 US|2004| 36450| 12.686|
 GB|2004| 7269| 2.487|
+-------+----+--------------+----------------------+

In [8]:
spark.sql("select py.Country, py.Year,py.total_quantity as py_tot_qty, \
                                      ly.total_quantity as ly_tot_qty, \
                                    py.total_sales_in_million as present_yr_sales_in_million, \
                                    ly.total_sales_in_million as past_yr_sales_in_million, \
            ((py.total_sales_in_million - ly.total_sales_in_million)/py.total_sales_in_million)*100 as SalesVariationtoLastYear \
          FROM agg_sales_2003 ly INNER JOIN agg_sales_2004 py  \
          ON ly.Country=py.Country").show()

+-------+----+----------+----------+---------------------------+------------------------+------------------------+
Country|Year|py_tot_qty|ly_tot_qty|present_yr_sales_in_million|past_yr_sales_in_million|SalesVariationtoLastYear|
+-------+----+----------+----------+---------------------------+------------------------+------------------------+
 AU|2004| 8696| 8152| 3.311| 3.804| -14.889761401389304|
 FR|2004| 6928| 9881| 2.299| 3.353| -45.84602000869945|
 CA|2004| 12090| 21727| 3.064| 6.152| -100.78328981723237|
 DE|2004| 6016| 6693| 1.982| 2.126| -7.2653884964682085|
 GB|2004| 7269| 9869| 2.487| 3.413| -37.233614796944096|
 US|2004| 36450| 67184| 12.686| 22.776| -79.53649692574491|
+-------+----+----------+----------+---------------------------+------------------------+------------------------+